In [1]:
# Using Aviation Edge and OpenSky APIs for historical data analysis

In [2]:
# Importing modules

import requests
import pandas as pd
import nest_asyncio
import asyncio
from datetime import datetime, timedelta
import time

In [11]:
# Setting up OpenSky

from python_opensky import OpenSky
nest_asyncio.apply()

class AirCanadaDataCollector:
    def __init__(self):
        self.flight_data = []
        self.historical_data = pd.DataFrame()   #Dataframe for storing Air Canada Flights
    
    async def collect_realtime_flights(self):
        """Collects current Air Canada flight data"""
        async with OpenSky() as opensky:
            
            # Getting most recent flight data
            states = await opensky.get_states()
            
            ac_flights = []

            # Filtering to get Air Canada flight data only
            for state in states.states:
                if state.callsign and state.callsign.strip().startswith('ACA'):
                    flight_info = {
                        'timestamp': datetime.now(),
                        'callsign': state.callsign.strip(),
                        'icao24': state.icao24,
                        'longitude': state.longitude,
                        'latitude': state.latitude,
                        'altitude': state.barometric_altitude,
                        'velocity': state.velocity,
                        'heading': state.true_track,
                        'vertical_rate': state.vertical_rate,
                        'on_ground': state.on_ground
                    }

                    # Adds the current Air Canada flight details to the list
                    ac_flights.append(flight_info)
            
            return pd.DataFrame(ac_flights)
    
    async def collect_historical_flights(self, start_time, end_time):
        """Collects historical flight data for a specific time range"""
        async with OpenSky() as opensky:
            # OpenSky provides historical data access
            flights = await opensky.get_flights_by_aircraft(
                begin=int(start_time.timestamp()),
                end=int(end_time.timestamp())
            )
            return flights

# Initializes collector object
collector = AirCanadaDataCollector()

# Collectes current flights
current_flights = await collector.collect_realtime_flights()
print(f"Collected {len(current_flights)} current Air Canada flights")
print(current_flights)

Collected 98 current Air Canada flights
                    timestamp callsign  icao24  longitude  latitude  altitude  \
0  2025-06-28 15:52:05.344948  ACA1307  c07c7b   -83.1113   36.0849  10660.38   
1  2025-06-28 15:52:05.344948  ACA1072  c07c7a   -79.7097   43.6166   9448.80   
2  2025-06-28 15:52:05.344948  ACA1040  c07e2c   -96.6950   41.3451  11277.60   
3  2025-06-28 15:52:05.344948   ACA732  c07e32   -73.9071   44.4442   6370.32   
4  2025-06-28 15:52:05.344948   ACA839  c0103e  -123.1824   49.1957       NaN   
..                        ...      ...     ...        ...       ...       ...   
93 2025-06-28 15:52:05.344948   ACA895  c04fbb   -71.4433   46.6397   9639.30   
94 2025-06-28 15:52:05.344948   ACA825  c04fe4   -76.9882   44.7990  10744.20   
95 2025-06-28 15:52:05.344948  ACA1048  c00700   -96.5559   43.3696  10675.62   
96 2025-06-28 15:52:05.344948   ACA608  c005c5   -67.4309   44.3757  10668.00   
97 2025-06-28 15:52:05.344948   ACA211  c00757  -113.9925   51.1112  